In [1]:
# imports & device configuration
import os
import re
import html
import itertools as it
import pickle
import hashlib
import warnings
from pathlib import Path
import numpy as np
import pandas as pd
import torch
from torch import nn
from torch.autograd import Function
from torch.utils.data import Dataset, DataLoader

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    default_data_collator,
    EarlyStoppingCallback
)

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.metrics import accuracy_score
from scipy.stats import ks_2samp, binomtest
from tqdm.auto import tqdm

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
tqdm.pandas(disable=False)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device}")

Using cpu


In [6]:
# Path definitions
dict_path = "dataset/FullDictionaries.csv"
amazon_appliances_path = "dataset/amazon_Appliances_5_jiadian.json"
amazon_fashion_path    = "dataset/AMAZON_FASHION_5_shishang.json"
amazon_beauty_path     = "dataset/All_Beauty_5_meizhuang.json"
amazon_pet_path        = "dataset/Pet_Supplies_5_sampled_2.json"
movie_path             = "dataset/Movie Reviews_train.tsv"
twitter1_path          = "dataset/train-twitter.tsv"
twitter2_path          = "dataset/test-twitter.tsv"

DRIVE_PATH = "output"
os.makedirs(DRIVE_PATH, exist_ok=True)

In [7]:
# Load Prepared Datasets

in_path = os.path.join(DRIVE_PATH, "datasets_prepared.pkl")
with open(in_path, "rb") as f:
    datasets = pickle.load(f)

In [8]:
# Load Train / Val / Test Splits

META_PATH = os.path.join(DRIVE_PATH, "all_results_meta.pkl")

if os.path.exists(META_PATH):
    with open(META_PATH, "rb") as f:
        all_results = pickle.load(f)

    for res in all_results.values():
        res["train_idx"] = np.array(res["train_idx"])
        res["val_idx"]   = np.array(res["val_idx"])
        res["test_idx"]   = np.array(res["test_idx"])
    print("all_results restored – keys:", list(all_results.keys()))
else:
    print("meta file not found")
    all_results = {}



all_results restored – keys: ['Amazon', 'Movie', 'Twitter']


In [9]:
# Sentence Length Normalization，normalize the 'warmth_scm' and competence_scm score by the number of tokens, converting raw sums into per-word averages
for ds in ["Amazon", "Movie", "Twitter"]:

    df = datasets[ds]["df"]
    df["num_tok"] = df[datasets[ds]["text_col"]].str.split().str.len().clip(lower=1)
    df["warmth_scm"]     /= df["num_tok"]
    df["competence_scm"] /= df["num_tok"]

In [ ]:
# Task-adaptive debias Amazon + Movie + Twitter

MAX_LEN          = 256
TRAIN_BATCH_SIZE = 128
EVAL_BATCH_SIZE  = 256

# Prepare a dataset class that holds inputs, labels, and bias scores
class DebiasDataset(torch.utils.data.Dataset):
    def __init__(self, enc, lab, w, c):
        self.enc, self.lab, self.w, self.c = enc, lab, w, c
    def __getitem__(self, i):
        d = {k: torch.tensor(v[i]) for k, v in self.enc.items()}
        d["labels"]         = torch.tensor(self.lab[i])
        d["warmth_scm"]     = torch.tensor(self.w[i], dtype=torch.float32)
        d["competence_scm"] = torch.tensor(self.c[i], dtype=torch.float32)
        return d
    def __len__(self): return len(self.lab)

# Define a Gradient Reversal Layer (GRL) for adversarial training
class GRL(Function):
    @staticmethod
    def forward(ctx, x, l):
      ctx.l = l;
      return x.view_as(x)
    @staticmethod
    def backward(ctx, g):
      return -ctx.l * g, None

# Convenience function to apply GRL
def grl(x, l=1.0): return GRL.apply(x, l)

# Build a small adversary network that predicts 3 bins: negative / zero / positive
class Adv3(nn.Module):
    def __init__(self, h):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(h, h//2), nn.ReLU(), nn.Linear(h//2, 3))
    def forward(self, x): return self.net(x)

# Custom Trainer to combine the main loss with adversarial loss
class DebiasTrainer(Trainer):
    def __init__(self, *a, adv_w, adv_c, alpha, **kw):
        # adv_w / adv_c: adversary models for warmth and competence
        # alpha: weight for the adversarial loss
        super().__init__(*a, **kw)
        self.adv_w, self.adv_c = adv_w.to(self.model.device), adv_c.to(self.model.device)
        self.alpha   = alpha
        self.ce_task = nn.CrossEntropyLoss()
        self.ce_adv  = nn.CrossEntropyLoss(reduction="none")

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        y = inputs.pop("labels")
        w = inputs.pop("warmth_scm")
        c = inputs.pop("competence_scm")


        out    = model(**inputs, output_hidden_states=True)

        # Compute main loss
        task_L = self.ce_task(out.logits, y)

        h    = out.hidden_states[-1][:, 0, :]

        # Create a per-example weight based on the magnitude of SCM scores
        mag  = (w.abs() + c.abs()) / 2
        rank = torch.argsort(torch.argsort(mag))
        lam  = torch.sqrt(rank.float()) / torch.sqrt(torch.tensor(len(rank) - 1, dtype=torch.float32))

        # Bin scores into 3 categories: neg / zero / pos
        def bin3(x): return (x > 0).long() + (x == 0).long()

        # Compute adversarial losses
        adv_L = 0.5 * (
            self.ce_adv(self.adv_w(grl(h)), bin3(w)) +
            self.ce_adv(self.adv_c(grl(h)), bin3(c))
        )

        # Total loss
        loss = task_L + self.alpha * (lam * adv_L).mean()
        return (loss, out) if return_outputs else loss

# Tokenize a list of raw texts in batches to prevent out-of-memory errors from processing all texts at once
def batch_tokenize(tok, texts, max_len, name, batch=1024):
    enc = {'input_ids': [], 'attention_mask': []}
    it = range(0, len(texts), batch)
    it = tqdm(it, desc=f"Tokenizing {name}", leave=False)

    # tokenize each batch
    for i in it:
        batch_text = [str(t) if isinstance(t, str) else "" for t in texts[i:i+batch]]
        out = tok(batch_text,
                  truncation=True,
                  padding='max_length',
                  max_length=max_len,
                  return_tensors=None)
        enc['input_ids'] += out['input_ids']
        enc['attention_mask'] += out['attention_mask']
    return enc

# Define an evaluation metrics function
def compute_metrics(pred):
    y_true = pred.label_ids
    y_pred = pred.predictions.argmax(-1)
    prec,rec,f1,_ = precision_recall_fscore_support(
        y_true, y_pred, average='macro', zero_division=0)
    return {'accuracy': accuracy_score(y_true, y_pred),
            'f1_macro': f1,
            'precision_macro': prec,
            'recall_macro': rec}

# Defined locations of baseline checkpoints for each dataset
BASE = {"Movie":   {"ep": 3, "step": 2928},
        "Twitter": {"ep": 5, "step":  810},
        "Amazon":  {"ep": 2, "step": 2206}}

# Different alpha values to try for adversarial weight
GRID_ALPHA = [0.10, 0.15, 0.20, 0.35, 0.5]

# 7. Main loop for each dataset and each alpha, train and evaluate
for ds in ["Twitter", "Amazon", "Movie", ]:
    base_ckpt = f"{DRIVE_PATH}/results_{ds}_ep{BASE[ds]['ep']}/checkpoint-{BASE[ds]['step']}"
    tok       = AutoTokenizer.from_pretrained(base_ckpt, use_fast=True, local_files_only=True)

    df        = datasets[ds]["df"];  txt = datasets[ds]["text_col"]
    tr_idx, va_idx = all_results[ds]["train_idx"], all_results[ds]["val_idx"]

    enc_tr = batch_tokenize(tok, df.loc[tr_idx, txt].tolist(), MAX_LEN, f"{ds}-tr")
    enc_va = batch_tokenize(tok, df.loc[va_idx, txt].tolist(), MAX_LEN, f"{ds}-va")

    ds_tr = DebiasDataset(enc_tr,
        (df.loc[tr_idx, 'label_5class'] - 1).values,
        df.loc[tr_idx, 'warmth_scm'].values,
        df.loc[tr_idx, 'competence_scm'].values)
    ds_va = DebiasDataset(enc_va,
        (df.loc[va_idx, 'label_5class'] - 1).values,
        df.loc[va_idx, 'warmth_scm'].values,
        df.loc[va_idx, 'competence_scm'].values)

    # Iterate over each alpha value
    for alpha in GRID_ALPHA:
        tag = f"a{int(alpha*100):03d}"
        print(f"\n=== {ds} | α={alpha:.2f} ===")

        mdl   = AutoModelForSequenceClassification.from_pretrained(base_ckpt, num_labels=5).to(device)
        adv_w = Adv3(mdl.config.hidden_size)
        adv_c = Adv3(mdl.config.hidden_size)

        steps_per_ep = max(1, len(tr_idx) // TRAIN_BATCH_SIZE)

        args = TrainingArguments(
            output_dir=f"{DRIVE_PATH}/results_debiased_{ds}_{tag}",
            num_train_epochs=5,
            per_device_train_batch_size=TRAIN_BATCH_SIZE,
            per_device_eval_batch_size=EVAL_BATCH_SIZE,
            save_steps = max(1, len(tr_idx) // TRAIN_BATCH_SIZE),
            save_total_limit=2,
            logging_steps=100,
            fp16=torch.cuda.is_available(),
            report_to="none",
            disable_tqdm=False,
            remove_unused_columns=False
        )

        # Initialize our custom trainer
        trainer = DebiasTrainer(
            model=mdl, args=args, tokenizer=tok,
            train_dataset=ds_tr, eval_dataset=ds_va,
            data_collator=default_data_collator,
            compute_metrics=compute_metrics,
            adv_w=adv_w, adv_c=adv_c, alpha=alpha,

        )
        trainer.train()

        best_ckpt = trainer.state.best_model_checkpoint or args.output_dir

        # Ensure it has a config.json, otherwise pick the latest sub-checkpoint
        if not os.path.isfile(os.path.join(best_ckpt, "config.json")):
            ck_sub = [d for d in os.listdir(best_ckpt) if d.startswith("checkpoint-")]
            best_ckpt = os.path.join(best_ckpt, sorted(ck_sub, key=lambda x:int(x.split('-')[1]))[-1])
        print("best checkpoint: ", best_ckpt.split("/")[-1])

        # Evaluate on the validation set and save metrics to CSV
        eval_args = TrainingArguments("/tmp/eval", per_device_eval_batch_size=EVAL_BATCH_SIZE,
                                      dataloader_pin_memory=False, report_to="none")
        mdl_best  = AutoModelForSequenceClassification.from_pretrained(best_ckpt).to(device)
        ev_tr     = Trainer(model=mdl_best, args=eval_args, tokenizer=tok,
                            data_collator=default_data_collator, compute_metrics=compute_metrics)
        metrics   = ev_tr.evaluate(ds_va)
        pd.DataFrame([metrics]).to_csv(f"{best_ckpt}/metrics_val.csv", index=False)
        print(f"{ds} | α={alpha:.2f} | eval_acc={metrics['eval_accuracy']:.4f}")

        # Clean up GPU memory before next run
        del trainer, mdl, mdl_best, ev_tr
        torch.cuda.empty_cache()

Tokenizing Twitter-tr:   0%|          | 0/21 [00:00<?, ?it/s]

Tokenizing Twitter-va:   0%|          | 0/3 [00:00<?, ?it/s]


=== Twitter | α=0.10 ===


<ipython-input-17-b17afefae5e7>:49: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `DebiasTrainer.__init__`. Use `processing_class` instead.
  super().__init__(*a, **kw)


Step,Training Loss
100,0.490500
200,0.522500
300,0.515300
400,0.495500
500,0.496000
600,0.476200
700,0.483100
800,0.480200


...

<ipython-input-17-b17afefae5e7>:49: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `DebiasTrainer.__init__`. Use `processing_class` instead.
  super().__init__(*a, **kw)


Step,Training Loss
100,1.539400
200,1.760400
300,1.781400
400,1.787700
500,1.830800
600,1.843800
700,1.835500
800,1.886900
900,1.902300
1000,1.885000


best checkpoint   checkpoint-4880


<ipython-input-17-b17afefae5e7>:155: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  ev_tr     = Trainer(model=mdl_best, args=eval_args, tokenizer=tok,


Movie | α=0.50 | eval_acc=0.6662


In [ ]:
# Fixed Debias  Amazon + Movie + Twitter Exactly like the original DebiasTrainer, except that every example’s debias weight lambda_i is fixed to 1.

class FixedDebiasTrainer(Trainer):

    def __init__(self, *a, adv_w, adv_c, alpha, **kw):
        super().__init__(*a, **kw)
        self.adv_w, self.adv_c = adv_w.to(self.model.device), adv_c.to(self.model.device)
        self.alpha   = alpha
        self.ce_task = nn.CrossEntropyLoss()
        self.ce_adv  = nn.CrossEntropyLoss(reduction="none")

    def compute_loss(self, model, inputs, return_outputs=False, **_):
        y = inputs.pop("labels")
        w = inputs.pop("warmth_scm")
        c = inputs.pop("competence_scm")

        out    = model(**inputs, output_hidden_states=True)
        task_L = self.ce_task(out.logits, y)

        h = out.hidden_states[-1][:, 0, :]
        lam = torch.ones_like(y, dtype=torch.float32)

        def bin3(x): return (x > 0).long() + (x == 0).long()
        adv_L = 0.5 * (
            self.ce_adv(self.adv_w(h), bin3(w)) +
            self.ce_adv(self.adv_c(h), bin3(c))
        )

        loss = task_L + self.alpha * (lam * adv_L).mean()
        return (loss, out) if return_outputs else loss


# Other setup
# Sentence-length normalization (skip if already done)

BASE = {"Movie":   {"ep": 3, "step": 2928},
        "Twitter": {"ep": 5, "step":  810},
        "Amazon":  {"ep": 2, "step": 2206}}

GRID_ALPHA = [0.10, 0.15, 0.20, 0.35, 0.5]

for ds in ["Amazon", "Movie", "Twitter"]:
    base_ckpt = f"{DRIVE_PATH}/results_{ds}_ep{BASE[ds]['ep']}/checkpoint-{BASE[ds]['step']}"
    tok       = AutoTokenizer.from_pretrained(base_ckpt, use_fast=True, local_files_only=True)

    df      = datasets[ds]["df"];   txt = datasets[ds]["text_col"]
    tr_idx, va_idx = all_results[ds]["train_idx"], all_results[ds]["val_idx"]

    enc_tr = batch_tokenize(tok, df.loc[tr_idx, txt].tolist(), MAX_LEN, f"{ds}-tr")
    enc_va = batch_tokenize(tok, df.loc[va_idx, txt].tolist(), MAX_LEN, f"{ds}-va")

    lab_tr = (df.loc[tr_idx, 'label_5class'] - 1).values
    lab_va = (df.loc[va_idx, 'label_5class'] - 1).values

    ds_tr = torch.utils.data.TensorDataset()
    def make_ds(enc, lab, w, c):
        class _D(torch.utils.data.Dataset):
            def __init__(self): pass
            def __len__(self): return len(lab)
            def __getitem__(self, i):
                d = {k: torch.tensor(v[i]) for k, v in enc.items()}
                d.update(labels=torch.tensor(lab[i]),
                         warmth_scm=torch.tensor(w[i], dtype=torch.float32),
                         competence_scm=torch.tensor(c[i], dtype=torch.float32))
                return d
        return _D()

    ds_tr = make_ds(enc_tr, lab_tr,
                    df.loc[tr_idx,'warmth_scm'].values,
                    df.loc[tr_idx,'competence_scm'].values)
    ds_va = make_ds(enc_va, lab_va,
                    df.loc[va_idx,'warmth_scm'].values,
                    df.loc[va_idx,'competence_scm'].values)

    steps_per_ep = max(1, len(tr_idx) // TRAIN_BATCH_SIZE)

    for alpha in GRID_ALPHA:
        tag = f"fixed_a{int(alpha*100):03d}"
        print(f"\n=== {ds} | {tag} ===")

        mdl   = AutoModelForSequenceClassification.from_pretrained(base_ckpt, num_labels=5).to(device)
        adv_w = nn.Sequential(nn.Linear(mdl.config.hidden_size, mdl.config.hidden_size//2),
                              nn.ReLU(),
                              nn.Linear(mdl.config.hidden_size//2, 3))
        adv_c = nn.Sequential(nn.Linear(mdl.config.hidden_size, mdl.config.hidden_size//2),
                              nn.ReLU(),
                              nn.Linear(mdl.config.hidden_size//2, 3))

        args = TrainingArguments(
            output_dir=f"{DRIVE_PATH}/results_debiased_{ds}_{tag}",
            num_train_epochs=5,
            per_device_train_batch_size=TRAIN_BATCH_SIZE,
            per_device_eval_batch_size=EVAL_BATCH_SIZE,
            save_steps=steps_per_ep,
            save_total_limit=2,
            logging_steps=100,
            fp16=torch.cuda.is_available(),
            report_to="none",
            disable_tqdm=False,
            remove_unused_columns=False
        )

        trainer = FixedDebiasTrainer(
            model=mdl, args=args, tokenizer=tok,
            train_dataset=ds_tr, eval_dataset=ds_va,
            data_collator=default_data_collator,
            compute_metrics=compute_metrics,
            adv_w=adv_w, adv_c=adv_c, alpha=alpha,
        )
        trainer.train()

        best_ckpt = trainer.state.best_model_checkpoint or args.output_dir
        print("saved to", best_ckpt)
        del trainer, mdl; torch.cuda.empty_cache()


Tokenizing Amazon-tr:   0%|          | 0/138 [00:00<?, ?it/s]

Tokenizing Amazon-va:   0%|          | 0/18 [00:00<?, ?it/s]


=== Amazon | fixed_a010 ===


Step,Training Loss
100,0.803800
200,0.708900
300,0.701200
400,0.649600
500,0.640800
600,0.640200
700,0.618200
800,0.650700
900,0.637700
1000,0.660700


  saved to /content/drive/MyDrive/SCM_Bias_Results/results_debiased_Amazon_fixed_a010

=== Amazon | fixed_a015 ===


...

  saved to /content/drive/MyDrive/SCM_Bias_Results/results_debiased_Twitter_fixed_a020

=== Twitter | fixed_a035 ===


Step,Training Loss
100,0.563800
200,0.453000
300,0.409900
400,0.383000
500,0.367100
600,0.346300
700,0.345600
800,0.341100


  saved to /content/drive/MyDrive/SCM_Bias_Results/results_debiased_Twitter_fixed_a035

=== Twitter | fixed_a050 ===


Step,Training Loss
100,0.625100
200,0.497800
300,0.454500
400,0.424300
500,0.410700
600,0.389500
700,0.387500
800,0.384900


  saved to /content/drive/MyDrive/SCM_Bias_Results/results_debiased_Twitter_fixed_a050
